In [1]:
import numpy as np
import string
from sklearn.metrics import accuracy_score,confusion_matrix
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import time

In [2]:
def preprocess(text):
    
    preprocessed_text = text.lower().replace('-', ' ')
    translation_table = str.maketrans('\n', ' ', string.punctuation+string.digits)
    preprocessed_text = text.translate(translation_table)
    
    return preprocessed_text

In [3]:
df_train = pd.read_csv('../../data/qald-7-train.csv')

In [4]:
df_train['questions'] = [preprocess(text) for text in df_train['questions'].values]

In [5]:
vectorizer = CountVectorizer()

In [6]:
X_train = vectorizer.fit_transform(df_train["questions"])
y_train = df_train["lang"]

In [7]:
naive_classifier = MultinomialNB(fit_prior=False,alpha=0.01)
naive_classifier.fit(X_train, y_train)

MultinomialNB(alpha=0.01, fit_prior=False)

In [10]:
l = ["../data/qald-3-test.csv","../data/qald-4-test.csv","../data/qald-5-test.csv","../data/qald-6-test.csv","../data/qald-7-test.csv","../data/qald-8-test.csv","../data/qald-9-test.csv"]

scores = []
times = []

for d in l:
    start_time = time.time()
    
    df_test = pd.read_csv("../"+d)
    
    # drop 'ru' and 'pt' lang data because langTag doesn't support them
    columns = [ 'ru', 'pt']
    for col in columns:
        df_test = df_test[df_test.lang!=col]
        
    df_test=df_test.dropna(subset=['keywords'])
    df_test['keywords'] = [preprocess(text) for text in df_test['keywords'].values]
    
    X_test = vectorizer.transform(df_test["keywords"])
    y_test = df_test["lang"]
    
    predictions = naive_classifier.predict(X_test)
    
    end_time = time.time()
    
    total_time = start_time-end_time
    
    scores.append(accuracy_score(y_test, predictions))
    
    times.append(c/df_test.shape[0])

In [11]:
scores

[0.702020202020202,
 0.7771428571428571,
 0.7708978328173375,
 0.7672090112640801,
 0.6744186046511628,
 0.4878048780487805,
 0.7033527696793003]

In [12]:
scores

[0.9966329966329966,
 0.9942857142857143,
 1.0,
 0.9925,
 0.9825581395348837,
 0.975609756097561,
 0.9744897959183674]

In [12]:
times

[5.485995450003781e-05,
 9.310517992292131e-05,
 0.00010088796586075066,
 4.0784496837324735e-05,
 0.00018945821495943292,
 0.0007948003164151819,
 2.3751321408908498e-05]